In [ ]:
# Uses a pre-trained language model (T5) for text generation
# Implements a Flask API for serving predictions
# Includes basic error handling and logging
# Uses configuration for model parameters
# Demonstrates multi-threading for running the Flask app in Colab

# it's recommended to build modular code 1.Separate the code into different modules (e.g., model.py, api.py, config.py)
#2.Use a main.py file to tie everything together

# industry level development and best practices
# Modularity: Split code into separate modules (e.g., model.py, api.py, config.py) for better organization.
# Configuration Management: Use tools like python-dotenv or hydra for cleaner config handling.
# Experiment Tracking: Integrate MLflow or Weights & Biases for model versioning and tracking.
# Input/Output Handling: Add robust input validation, preprocessing, and output postprocessing.
# Caching: Use Redis or Memcached to cache frequent requests.
# Monitoring: Add comprehensive logging and monitoring with Prometheus or Grafana.
# Testing: Implement unit tests, integration tests, and CI/CD pipelines.
# Performance: Optimize inference with batching or model quantization.
# Scalability: Design for horizontal scaling (e.g., Kubernetes).
# Documentation: Add detailed docstrings and API documentation.


!pip install transformers torch flask datasets pyyaml

import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from flask import Flask, request, jsonify
import logging
import os
import yaml
from threading import Thread
import time
import requests

# Logging setup
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

# Configuration
config = {
    'api_key': 'YOUR_SIMULATED_API_KEY',
    'model_name': 't5-base',
    'max_length': 100,
    'temperature': 0.8,
    'top_k': 50,
    'top_p': 0.95,
    'num_beams': 5,
    'no_repeat_ngram_size': 2,
    'repetition_penalty': 1.5,
    'host': '0.0.0.0',
    'port': 5001
}

API_KEY = config['api_key']

class ContentGenerator:
    def __init__(self, model_name=config['model_name']):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")

    def generate_content(self, product_description, target_audience, desired_tone, max_length=config['max_length']):
        try:
            prompt = f"Write a {desired_tone} marketing copy for {product_description} targeting {target_audience}."
            input_ids = self.tokenizer(prompt, return_tensors="pt").input_ids.to(self.model.device)
            output = self.model.generate(
                input_ids,
                max_length=max_length,
                temperature=config['temperature'],
                top_k=config['top_k'],
                top_p=config['top_p'],
                num_beams=config['num_beams'],
                no_repeat_ngram_size=config['no_repeat_ngram_size'],
                repetition_penalty=config['repetition_penalty']
            )
            generated_text = self.tokenizer.decode(output[0], skip_special_tokens=True)
            return generated_text
        except Exception as e:
            logger.error(f"Error generating content: {e}")
            raise e

# Flask app setup
app = Flask(__name__)
content_generator = ContentGenerator()

def authenticate(request):
    api_key = request.headers.get("X-Api-Key")
    return api_key == API_KEY

@app.route("/generate", methods=["POST"])
def generate():
    if not authenticate(request):
        return jsonify({"error": "Unauthorized"}), 401

    try:
        data = request.get_json()
        product_description = data.get("product_description")
        target_audience = data.get("target_audience")
        desired_tone = data.get("desired_tone")

        if not all([product_description, target_audience, desired_tone]):
            return jsonify({"error": "Missing required parameters"}), 400

        content = content_generator.generate_content(product_description, target_audience, desired_tone)

        logger.info(f"Generated content for product: {product_description}, audience: {target_audience}, tone: {desired_tone}")

        return jsonify({"content": content})
    except Exception as e:
        logger.error(f"Error generating content: {e}")
        return jsonify({"error": str(e)}), 500

# Run the app in a separate thread
def run_app():
    app.run(debug=False, host=config['host'], port=config['port'])

thread = Thread(target=run_app)
thread.start()

# Wait for the app to start
time.sleep(5)

# Test the API
headers = {"X-Api-Key": API_KEY}
data = {"product_description": "a new AI-powered chatbot", "target_audience": "businesses", "desired_tone": "informative"}

response = requests.post(f"http://{config['host']}:{config['port']}/generate", headers=headers, json=data)

if response.status_code == 200:
    print("Generated content:", response.json()["content"])
else:
    print("Error:", response.json())

# Additional product examples to test
product_examples = [
    "a sustainable clothing line for eco-conscious consumers",
    "a new AI-powered fitness tracker",
    "an online course for learning data science",
    "a subscription box for organic pet food",
    "a productivity app for managing tasks and projects",
]

for product_description in product_examples:
    data = {"product_description": product_description, "target_audience": "young adults", "desired_tone": "persuasive"}
    response = requests.post(f"http://{config['host']}:{config['port']}/generate", headers=headers, json=data)
    print(f"\nProduct: {product_description}")
    print("Generated content:", response.json().get("content", "Error: " + str(response.json())))

# Stop the Flask app
import os
import signal
os.kill(os.getpid(), signal.SIGINT)
